# The “New Style” Class





## The “New Style” Class Model


In release 2.2, Python introduced a new flavor of classes, known as *new-style* classes;
classes following the original and traditional model became known as *classic* classes
when compared to the new kind. In 3.X the class story has merged, but it remains split
for Python 2.X users and code:

  + In *Python 3.X*, all classes are automatically what were formerly called “new style,”
    whether they explicitly inherit from `object` or not. Coding the `object` superclass is
    optional and implied.

  + In *Python 2.X*, classes must explicitly inherit from `object` (or another built-in type)
    to be considered “new style” and enable and obtain all new-style behavior. Classes
    without this are “classic.”

Because all classes are automatically new-style in 3.X, the features of new-style classes
are simply normal class features in that line. I’ve opted to keep their descriptions in this
section separate, however, in deference to users of Python 2.X code—classes in such
code acquire new-style features and behavior only when they are derived from `object`.

In other words, when Python 3.X users see descriptions of “new style” topics in this
book, they should take them to be descriptions of existing properties of their classes.
For 2.X readers, these are a set of optional changes and extensions that you may choose
to enable or not, unless the code you must use already employs them.

In Python 2.X, the identifying *syntactic* difference for new-style classes is that they are
derived from either a built-in type, such as `list`, or a special built-in class known as
`object`. The built-in name `object` is provided to serve as a superclass for new-style
classes if no other built-in type is appropriate to use:

```python
    class newstyle(object):                 # 2.X explicit new-style derivation
    ...normal class code...                 # Not required in 3.X: automatic
```

Any class derived from `object`, or any other built-in type, is automatically treated as a
new-style class. That is, as long as a built-in type is somewhere in its superclass tree, a
2.X class acquires new-style class behavior and extensions. Classes not derived from
built-ins such as object are considered classic.

### Just How New Is New-Style?


As we’ll see, new-style classes come with profound differences that impact programs
broadly, especially when code leverages their added advanced features. In fact, at least
in terms of its OOP support, these changes on some levels transform Python into a
*different language altogether*—one that’s mandated in the 3.X line, one that’s optional
in 2.X only if ignored by every programmer, and one that borrows much more from
(and is often as complex as) other languages in this domain.

New-style classes stem in part from an attempt to merge the notion of *class* with that
of *type* around the time of Python 2.2, though they went unnoticed by many until they
were escalated to required knowledge in 3.X. You’ll need to judge the success of that
merging for yourself, but as we’ll see, there are still distinctions in the model—now
between *class* and *metaclass*—and one of its side effects is to make normal classes more
powerful but also substantially more complex. The new-style inheritance algorithm
formalized in [Chapter 40](), for example, grows in complexity by at least a factor of 2.

Still, some programmers using straightforward application code may notice only slight
divergence from traditional “classic” classes. After all, we’ve managed to get to this
point in this book writing substantial class examples, with mostly just passing mentions
of this change. Moreover, the classic class model still available in 2.X works exactly as
it has for some two decades.

However, because they modify core class behaviors, new-style classes had to be introduced 
in Python 2.X as a distinct tool so as to avoid impacting any existing code that
depends on the prior model. For example, some subtle differences, such as diamond
pattern inheritance search and the interaction of built-in operations and managed 
attribute methods such as `__getattr__` can cause some existing code to fail if left 
unchanged. Using optional extensions in the new model such as slots can have the same
effect.

The class model split is removed in Python 3.X, which *mandates* new-style classes, but
it still exists for readers using 2.X, or reusing the vast amount of existing 2.X code in
production use. Because this has been an optional extension in 2.X, code written for
that line may use either class model.

The next two top-level sections provide overviews of the ways in which new-style
classes differ and the new tools they provide. These topics represent potential changes
to some Python 2.X readers, but simply additional advanced class topics to many
Python 3.X readers. If you’re in the latter group, you’ll find full coverage here, though
some of it is presented in the context of changes—which you can accept as features,
but only if you never must deal with any of the millions of lines of existing 2.X code.

## New-Style Class Changes


New-style classes differ from classic classes in a number of ways, some of which are
subtle but can impact both existing 2.X code and common coding styles. As preview
and summary, here are some of the most prominent ways they differ:

  + *Attribute fetch for built-ins: instance skipped*  
    The `__getattr__` and `__getattribute__` generic attribute interception methods are
    still run for attributes accessed by explicit name, but no longer for attributes 
    implicitly fetched by built-in operations. They are not called for `__X__` operator 
    overloading method names in built-in contexts only—the search for such names begins
    at classes, not instances. This breaks or complicates objects that serve as *proxies*
    for another object’s interface, if wrapped objects implement operator overloading.
    Such methods must be redefined for the sake of differing built-ins dispatch in 
    new-style classes.

  + *Classes and types merged: type testing*  
    Classes are now types, and types are now classes. In fact, the two are essentially
    synonyms, though the *metaclasses* that now subsume types are still somewhat distinct 
    from normal classes. The `type(I)` built-in returns the class an instance is made
    from, instead of a generic instance type, and is normally the same as `I.__class__`.
    Moreover, classes are instances of the `type` class, and `type` may be subclassed to
    customize class creation with metaclasses coded with `class` statements. This can
    impact code that tests types or otherwise relies on the prior type model.

  + *Automatic object root class: defaults*  
    All new-style classes (and hence types) inherit from `object`, which comes with a
    small set of default operator overloading methods (e.g., `__repr__`). In 3.X, this class
    is added automatically above the user-defined root (i.e., *topmost*) classes in a tree,
    and need not be listed as a superclass explicitly. This can affect code that assumes
    the absence of method defaults and root classes.

  + *Inheritance search order: MRO and diamonds*  
    Diamond patterns of multiple inheritance have a slightly different search order—
    roughly, at diamonds they are searched across before up, and more breadth-first
    than depth-first. This attribute search order, known as the MRO, can be traced
    with a new `__mro__` attribute available on new-style classes. The new search order
    largely applies only to diamond class trees, though the new model’s implied
    object root itself forms a diamond in all multiple inheritance trees. Code that relies
    on the prior order will not work the same.

  + *Inheritance algorithm: [Chapter 40]()*  
    The algorithm used for inheritance in new-style classes is substantially more complex 
    than the depth-first model of classic classes, incorporating special cases for
    descriptors, metaclasses, and built-ins. We won’t be able to formalize this until
    [Chapter 40]() after we’ve studied metaclasses and descriptors in more depth, but it
    can impact code that does not anticipate its extra convolutions.

  + *New advanced tools: code impacts*  
    New-style classes have a set of new class tools, including *slots*, *properties*, *descriptors*, `super`, and the `__getattribute__` method. Most of these have very specific
    tool-building purposes. Their use can also impact or break existing code, though;
    slots, for example, sometimes prevent creation of an instance namespace dictionary
    altogether, and generic attribute handlers may require different coding.

We’ll explore the *extensions* noted in the last of these items in a later top-level section
of its own, and will defer formal inheritance algorithm coverage until [Chapter 40]() as
noted. Because the other items on this list have the potential to break traditional Python
code, though, let’s take a closer look at each in turn here.


### Attribute Fetch for Built-ins Skips Instances


We introduced this new-style class change in sidebars in both [Chapter 28]() and [Chapter 31]() 
because of their impact on prior examples and topics. In new-style classes (and
hence all classes in 3.X), the generic instance attribute interception methods 
`__getattr__` and `__getattribute__` are no longer called by built-in operations for `__X__` 
operator overloading method names—the search for such names begins at classes, not
instances. Attributes accessed by explicit name, however, are routed through these
methods, even if they are `__X__` names. Hence, this is primarily a change to the behavior
of built-in operations.

More formally, if a class defines a `__getitem__` index overload method and `X` is an 
instance of this class, then an index expression like X[I]  is roughly equivalent to 
`X.__getitem__(I)` for classic classes, but `type(X).__getitem__(X, I)` for new-style classes—the
latter beginning its search in the class, and thus skipping a `__getattr__` step from the
instance for an undefined name.

Technically, this method search for built-in operations like `X[I]`  uses normal inheritance 
beginning at the class level, and inspects only the *namespace dictionaries* of all
the classes from which `X` derives—a distinction that can matter in the *metaclass* model
we’ll meet later in this section and focus on in [Chapter 40](), where classes may acquire
behavior differently. The instance, however, is omitted by built-ins’ search.

#### Why the lookup change?


You can find formal rationales for this change elsewhere; this book is disinclined to
parrot justifications for a change that breaks many working programs. But this is 
imagined as both an *optimization* path and a solution to a seemingly obscure *call pattern* 
issue. The former rationale is supported by the frequency of built-in operations. If
every `+`, for example, requires extra steps at the instance, it can degrade program speed
—especially so given the new-style model’s many attribute-level extensions.

The latter rationale is more obscure, and is described in Python manuals; in short, it
reflects a conundrum introduced by the *metaclass* model. Because classes are now 
instances of metaclasses, and because metaclasses can define built-in operator methods
to process the classes they generate, a method call run for a class must skip the class
itself and look one level higher to pick up a method that processes the class, rather than
selecting the class’s own version. Its own version would result in an unbound method
call, because the class’s own method processes lower instances. This is just the usual
unbound method model we discussed in the prior section, but is potentially aggravated
by the fact that classes can acquire type behavior from metaclasses too.

As a result, because classes are both types and instances in their own right, all instances
are skipped for built-in operation method lookup. This is supposedly applied to normal
instances for uniformity and consistency, but both non-built-in names and direct and
explicit calls to built-in names still check the instance anyhow. Though perhaps a 
consequence of the new-style class model, to some this may seem a solution arrived at for
the sake of a usage pattern that was more artificial and obscure than the widely used
one it broke. Its role as optimization path seems more defensible, but also not without
repercussions.

In particular, this has potentially broad implications for the *delegation*-based classes,
often known as ***proxy classes***, when embedded objects implement operator overloading. 
In new-style classes, such a proxy object’s class must generally *redefine* any such
names to catch and delegate, either manually or with tools. The net effect is to either
significantly complicate or wholly obviate an *entire category of programs*. We explored
delegation in [Chapter 28]() and [Chapter 31](); it’s a common pattern used to augment or
adapt another class’s interface—to add validation, tracing, timing, and many other
sorts of logic. Though proxies may be more the exception than the rule in typical Python
code, many Python programs depend upon them.

#### Implications for attribute interception


In simple terms, and run in Python 2.X to show how new-style classes differ, indexing
and prints are routed to `__getattr__` in traditional classes, but not for new-style classes,
where printing uses a default:

```python
    >>> class C:
                data = 'spam'
                def __getattr__(self, name):  # Classic in 2.X: catches built-ins
                    print(name)
                    return getattr(self.data, name)
    >>> X = C()
    >>> X[0]
    __getitem__
    's'
    >>> print(X) # Classic doesn't inherit default
    __str__
    spam
```

In [ ]:
class C(object):  # New-style in 2.X and 3.X
    data = 'spam'
    def __getattr__(self, name):  # Classic in 2.X: catches built-ins
        print(name)
        return getattr(self.data, name)

X = C() # Built-ins not routed to getattr
X[0]

TypeError: 'C' object is not subscriptable

In [ ]:
print(X)

Though apparently rationalized in the name of class metaclass methods and optimizing
built-in operations, this divergence is not addressed by special-casing normal instances
having a `__getattr__`, and applies only to built-in operations—not to normally named
methods, or explicit calls to built-in methods by name:

```python
    >>> class C: pass                               # 2.X classic class
    >>> X = C()
    >>> X.normal = lambda: 99
    >>> X.normal()
    99
    >>> X.__add__ = lambda(y): 88 + y
    >>> X.__add__(1)
    89
    >>> X + 1
    89
```

In [ ]:
class C(object): pass                           # 2.X/3.X new-style class
X = C()
X.normal = lambda: 99
X.normal()                                      # Normals still from instance

99

In [ ]:
X.__add__ = lambda y: 88 + y
X.__add__(1)                                    # Ditto for explicit built-in names

89

In [ ]:
X + 1

TypeError: unsupported operand type(s) for +: 'C' and 'int'

This behavior winds up being inherited by the `__getattr__` attribute interception
method:

In [ ]:
class C(object):
    def __getattr__(self, name): print(name)

X = C()
X.normal                            # Normal names are still routed to getattr

normal


In [ ]:
X.__add__                           # Direct calls by name are too, but expressions are not!

__add__


In [ ]:
X + 1

TypeError: unsupported operand type(s) for +: 'C' and 'int'

#### Proxy coding requirements


In a more realistic delegation scenario, this means that built-in operations like 
expressions no longer work the same as their traditional direct-call equivalent. 
Asymmetrically, direct calls to built-in method names still work, but equivalent expressions do
not because through-type calls fail for names not at the class level and above. In other
words, this distinction arises in *built-in operations only*; explicit fetches run correctly:

In [ ]:
class C(object):
    data = 'spam'
    def __getattr__(self, name):
        print('getattr: ' + name)
        return getattr(self.data, name)

X = C()
X.__getitem__(1)                        # Traditional mapping works but new-style's does not

getattr: __getitem__


'p'

In [ ]:
X[1]

TypeError: 'C' object is not subscriptable

In [ ]:
type(X).__getitem__(X, 1)

AttributeError: type object 'C' has no attribute '__getitem__'

In [ ]:
X.__add__('eggs')                   # Ditto for +: instance skipped for expression only

getattr: __add__


'spameggs'

In [ ]:
X + 'eggs'

TypeError: unsupported operand type(s) for +: 'C' and 'str'

In [ ]:
type(X).__add__(X, 'eggs')

AttributeError: type object 'C' has no attribute '__add__'

The net effect: to code a proxy of an object whose interface may in part be invoked by
built-in operations, new-style classes require both `__getattr__` for normal names, as
well as method *redefinitions* for all names accessed by built-in operations—whether
coded manually, obtained from superclasses, or generated by tools. When redefinitions
are so incorporated, calls through *both* instances and types are equivalent to built-in
operations, though redefined names are no longer routed to the generic `__getattr__`
undefined name handler, even for explicit name calls:

In [ ]:
class C(object):                                        # New-style: 3.X and 2.X
    data = 'spam'
    def __getattr__(self, name):                        # Catch normal names
        print('getattr: ' + name)
        return getattr(self.data, name)
    def __getitem__(self, i):                           # Redefine built-ins
        print('getitem: ' + str(i))
        return self.data[i]                             # Run expr or getattr
    def __add__(self, other):
        print('add: ' + other)
        return getattr(self.data, '__add__')(other)

X = C()
X.upper

getattr: upper


<function str.upper()>

In [ ]:
X.upper()

getattr: upper


'SPAM'

In [ ]:
X[1]                                        # Built-in operation (implicit)

getitem: 1


'p'

In [ ]:
X.__getitem__(1)                            # Traditional equivalence (explicit)

getitem: 1


'p'

In [ ]:
type(X).__getitem__(X, 1)                   # New-style equivalence

getitem: 1


'p'

In [ ]:
type(X).__getitem__(X, 1)                   # New-style equivalence

getitem: 1


'p'

In [ ]:
X + 'eggs'                                  # Ditto for + and others

add: eggs


'spameggs'

In [ ]:
X.__add__('eggs')

add: eggs


'spameggs'

In [ ]:
type(X).__add__(X, 'eggs')

add: eggs


'spameggs'

#### For more details


We will revisit this change in [Chapter 40]() on metaclasses, and by example in the contexts
of attribute management in [Chapter 38]() and privacy decorators in [Chapter 39](). In the
latter of these, we’ll also explore coding structures for providing proxies with the 
required operator methods generically—it’s not an impossible task, and may need to be
coded just once if done well. For more of the sort of code influenced by this issue, see
those later chapters, as well as the earlier examples in [Chapter 28]() and [Chapter 31]().

Because we’ll expand on this issue later in the book, we’ll cut the coverage short here.
For external links and pointers on this issue, though, see the following (along with your
local search engine):

  + *Python Issue 643841*: this issue has been discussed widely, but its most official
    history seems to be documented at http://bugs.python.org/issue643841 . There, it
    was raised as a concern for real programs and escalated to be addressed, but a
    proposed library remedy or broader change in Python was struck down in favor of
    a simple documentation change to describe the new mandated behavior.

  + *Tool recipes*: also see http://code.activestate.com/recipes/252151 , an Active State
    Python recipe that describes a tool that automatically fills in special method names
    as generic call dispatchers in a proxy class created with metaclass techniques 
    introduced later in this section. This tool still must ask you to pass in the operator
    method names that a wrapped object may implement, though (it must, as interface
    components of a wrapped object may be inherited from arbitrary sources).

  + *Other approaches*: a web search today will uncover numerous additional tools that
    similarly populate proxy classes with overloading methods; it’s a widespread concern! Again, in [Chapter 39](), we’ll also see how to code straightforward and general
    superclasses once that provide the required methods or attributes as *mix-ins*,
    without metaclasses, redundant code generation, or similarly complex techniques.

This story may evolve over time, of course, but has been an issue for many years. As
this stands today, classic class proxies for objects that do any operator overloading are
effectively broken as new-style classes. Such classes in both 2.X and 3.X require coding
or generating wrappers for all the implicitly invoked operator methods a wrapped object 
may support. This is not ideal for such programs—some proxies may require dozens of 
wrapper methods (potentially over 50!)—but reflects, or is at least an artifact of,
the design goals of new-style class developers.

### Type Model Changes


On to our next new-style change: depending on your assessment, in new-style classes
the distinction between *type* and *class* has either been greatly muted or has vanished
entirely. Specifically:

- *Classes are types*

    The `type` object generates classes as its instances, and classes generate instances of
    themselves. Both are considered types, because they generate instances. In fact,
    there is no real difference between built-in types like lists and strings and 
    user-defined types coded as classes. This is why we can subclass built-in types, as shown
    earlier in this section—a subclass of a built-in type such as `list` qualifies as a 
    newstyle class and becomes a new user-defined type.

- *Types are classes*

    New class-generating types may be coded in Python as the *metaclasses* we’ll meet
    later in this section—user-defined type subclasses that are coded with normal
    class statements, and control creation of the classes that are their instances. As
    we’ll see, metaclasses are both class and type, though they are distinct enough to
    support a reasonable argument that the prior type/class dichotomy has become
    one of metaclass/class, perhaps at the cost of added complexity in normal classes.

Besides allowing us to subclass built-in types and code metaclasses, one of the most
practical contexts where this type/class merging becomes most obvious is when we do
explicit type testing. With Python 2.X’s classic classes, the type of a class instance is a
generic “instance,” but the types of built-in objects are more specific.

With new-style classes in 2.X, the type of a class instance is the class it’s created
from, since classes are simply user-defined types—the type of an instance is its class,
and the type of a user-defined class is the same as the type of a built-in object type.
Classes have a `__class__` attribute now, too, because they are instances of `type`.

The same is true for all classes in Python 3.X, since all classes are automatically new-style, 
even if they have no explicit superclasses. In fact, the distinction between built-in types 
and user-defined class types seems to melt away altogether in 3.X:


In [ ]:
class C: pass

I = C()                         # All classes are new-style in 3.X
type(I), I.__class__            # Type of instance is class it's made from

(__main__.C, __main__.C)

In [ ]:
type(C), C.__class__            # Class is a type, and type is a class

(type, type)

In [ ]:
type([1, 2, 3]), [1, 2, 3].__class__

(list, list)

In [ ]:
type(list), list.__class__ # Classes and built-in types work the same

(type, type)

As you can see, in 3.X classes are types, but types are also classes. Technically, each
class is generated by a *metaclass*—a class that is normally either *type* itself, or a subclass
of it customized to augment or manage generated classes. Besides impacting code that
does type testing, this turns out to be an important hook for tool developers. We’ll talk
more about metaclasses later in this section, and again in more detail in [Chapter 40]().

#### Implications for type testing


Besides providing for built-in type customization and metaclass hooks, the merging of
classes and types in the new-style class model can impact code that does type testing.
In Python 3.X, for example, the types of class instances compare directly and meaningfully, 
and in the same way as built-in type objects. This follows from the fact that
classes are now types, and an instance’s type is the instance’s class:

In [ ]:
class C: pass
class D: pass

c, d = C(), D()
type(c) == type(d)                      # 3.X: compares the instances' classes

False

In [ ]:
type(c), type(d)

(__main__.C, __main__.D)

In [ ]:
c.__class__, d.__class__

(__main__.C, __main__.D)

In [ ]:
c1, c2 = C(), C()
type(c1) == type(c2)

True

### All Classes Derive from “object”


Another ramification of the type change in the new-style class model is that because all
classes derive (inherit) from the class `object` either implicitly or explicitly, and because
all types are now classes, every object derives from the `object` built-in class, whether
directly or through a superclass. Consider the following interaction in Python 3.X:

In [ ]:
class C: pass # For new-style classes
X = C()
type(X), type(C) # Type is class instance was created from

(__main__.C, type)

As before, the type of a class *instance* is the class it was made from, and the type of a
*class* is the `type` class because classes and types have merged. It is also true, though,
that the instance and class are both derived from the built-in `object` class and type, an
implicit or explicit superclass of every class:

In [ ]:
isinstance(X, object), isinstance(C, object)        # Classes always inherit from object

(True, True)

In fact, `type` itself derives from `object`, and `object` derives from `type`, even though the
two are different objects—a circular relationship that caps the object model and stems
from the fact that types are classes that generate classes:

In [ ]:
type(type)                                          # All classes are types, and vice versa

type

In [ ]:
type(object)

type

In [ ]:
isinstance(type, object)                    # All classes derive from object, even type

True

In [ ]:
isinstance(object, type)                    # Types make classes, and type is a class

True

In [ ]:
type is object

False

#### Implications for defaults


The preceding may seem obscure, but this model has a number of practical implications. 
For one thing, it means that we sometimes must be aware of the method defaults
that come with the explicit or implicit *object* root class in new-style classes only:

In [ ]:
print(dir(object))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [ ]:
class C: pass
C.__bases__ # Classic classes do not inherit from object

(object,)

In [ ]:
X = C()
X.__repr__

<method-wrapper '__repr__' of C object at 0x000002B0DDA8D0D0>

This model also makes for fewer special cases than the prior type/class distinction of
classic classes, and it allows us to write code that can safely assume and use an
`object` superclass (e.g., by assuming it as an “anchor” in some `super` built-in roles 
described ahead, and by passing it method calls to invoke default behavior). We’ll see
examples of the latter later in the book; for now, let’s move on to explore the last major
new-style change.

### Diamond Inheritance Change


Our final new-style class model change is also one of its most visible: its slightly different
inheritance search order for so-called ***diamond pattern multiple inheritance trees***—a
tree pattern in which more than one superclass leads to the same higher superclass
further above (and whose name comes from the diamond shape of the tree if you sketch
out—a square resting on one of its corners).

The diamond pattern is a fairly advanced design concept, only occurs in multiple inheritance 
trees, and tends to be coded rarely in Python practice, so we won’t cover this
topic in full depth. In short, though, the differing search orders were introduced briefly
in the prior section’s multiple inheritance coverage:

- *For classic classes (the default in 2.X): DFLR*

    The inheritance search path is strictly depth first, and then left to right—Python
    climbs all the way to the top, hugging the left side of the tree, before it backs up
    and begins to look further to the right. This search order is known as ***DFLR*** for the
    first letters in its path’s directions.

- *For new-style classes (optional in 2.X and automatic in 3.X): MRO*

    The inheritance search path is more breadth-first in diamond cases—Python first
    looks in any superclasses to the right of the one just searched before ascending to
    the common superclass at the top. In other words, this search proceeds across by
    levels before moving up. This search order is called the new-style ***MRO*** for “method
    resolution order” (and often just MRO for short when used in contrast with the
    DFLR order). Despite the name, this is used for all attributes in Python, not just
    methods.

The new-style MRO algorithm is a bit more complex than just described—and we’ll
expand on it a bit more formally later—but this is as much as many programmers need
to know. Still, it has both important benefits for new-style class code, as well as 
program-breaking potential for existing classic class code.

For example, the new-style MRO allows lower superclasses to overload attributes of
higher superclasses, regardless of the sort of multiple inheritance trees they are mixed
into. Moreover, the new-style search rule avoids visiting the same superclass more than
once when it is accessible from multiple subclasses. It’s arguably better than DFLR, but
applies to a small subset of Python user code; as we’ll see, though, the new-style class
model *itself* makes diamonds much more common, and the MRO more important.

At the same time, the new MRO will locate attributes differently, creating a potential
incompatibility for 2.X classic classes. Let’s move on to some code to see how its 
differences pan out in practice.

#### Implications for diamond inheritance trees


To illustrate how the new-style MRO search differs, consider this simplistic incarnation
of the diamond multiple inheritance pattern for *classic classes*. Here, `D`’s superclasses
`B` and `C` both lead to the same common ancestor, `A`:

```python
    >>> class A:  attr = 1 # Classic (Python 2.X)
    >>> class B(A):  pass # B and C both lead to A
    >>> class C(A):  attr = 2
    >>> class D(B, C): pass # Tries A before C
    >>> x = D()
    >>> x.attr # Searches x, D, B, A
    1
```

The attribute `x.attr` here is found in superclass `A`, because with classic classes, the
inheritance search climbs as high as it can before backing up and moving right. The
full DFLR search order would visit `x`, `D`, `B`, `A`, `C`, and then `A`. For this attribute, the search
stops as soon as `attr` is found in `A`, above `B`.

However, with *new-style classes* derived from a built-in like `object` (and all classes in
3.X), the search order is different: Python looks in `C` to the right of `B`, before trying `A`
above `B`. The full MRO search order would visit `x`, `D`, `B`, `C`, and then `A`. For this attribute,
the search stops as soon as `attr` is found in C:

In [ ]:
class A(object): attr = 1                   # New-style ("object" not required in 3.X)
class B(A): pass
class C(A): attr = 2
class D(B, C):  pass                        # Tries C before A

x = D()
x.attr                                      # Searches x, D, B, C

2

This change in the inheritance search procedure is based upon the assumption that if
you mix in `C` lower in the tree, you probably intend to grab its attributes in preference
to `A`’s. It also assumes that `C` is always intended to override `A`’s attributes in all contexts,
which is probably true when it’s used standalone but may not be when it’s mixed into
a diamond with classic classes—you might not even know that `C` may be mixed in like
this when you code it.

Since it is most likely that the programmer meant that `C` should override `A` in this case,
though, new-style classes visit `C` first. Otherwise, `C` could be essentially pointless in a
diamond context for any names in `A` too—it could not customize `A` and would be used
only for names unique to `C`.

#### Explicit conflict resolution


Of course, the problem with assumptions is that they assume things! If this search order
deviation seems too subtle to remember, or if you want more control over the search
process, you can always force the selection of an attribute from anywhere in the tree
by assigning or otherwise naming the one you want at the place where the classes are
mixed together. The following, for example, chooses new-style order in a classic class
by resolving the choice explicitly:

```python
    >>> class A:  attr = 1                      # Classic
    >>> class B(A): pass
    >>> class C(A): attr = 2
    >>> class D(B, C): attr = C.attr            # <== Choose C, to the right
    
    >>> x = D()
    >>> x.attr                                  # Works like new-style (all 3.X)
    2
```

Here, a tree of classic classes is emulating the search order of new-style classes for a
specific attribute: the assignment to the attribute in `D` picks the version in `C`, thereby
subverting the normal inheritance search path (`D.attr` will be lowest in the tree). 
Newstyle classes can similarly emulate classic classes by choosing the higher version of the
target attribute at the place where the classes are mixed together:

In [ ]:
class A(object): attr = 1                       # New-style
class B(A):  pass
class C(A): attr = 2
class D(B, C):  attr = B.attr                   # <== Choose A.attr, above

x = D()
x.attr                                          # Works like classic (default 2.X)

1

If you are willing to always resolve conflicts like this, you may be able to largely ignore
the search order difference and not rely on assumptions about what you meant when
you coded your classes.

Naturally, attributes picked this way can also be method functions—methods are normal, 
assignable attributes that happen to reference callable function objects:

```bash
    >>> class A:
            def meth(s): print('A.meth')

    >>> class C(A):
            def meth(s): print('C.meth')
            
    >>> class B(A):
            pass

    >>> class D(B, C): pass                         # Use default search order

    >>> x = D()                                     # Will vary per class type
    >>> x.meth()                                    # Defaults to classic order in 2.X
    A.meth

    >>> class D(B, C): meth = C.meth                # <== Pick C's method: new-style (and 3.X)
    >>> x = D()
    >>> x.meth()
    C.meth

    >>> class D(B, C): meth = B.meth                # <== Pick B's method: classic
    >>> x = D()
    >>> x.meth()
    A.meth
```

In [ ]:
class A:
    def meth(s): print('A.meth')

class C(A):
    def meth(s): print('C.meth')

class B(A):
    pass

class D(B, C): pass                         # Use default search order

x = D()                                     # Will vary per class type
x.meth()                                    # Defaults to classic order in 2.X

C.meth


Here, we select methods by explicitly assigning to names lower in the tree. We might
also simply call the desired class explicitly; in practice, this pattern might be more
common, especially for things like constructors:

```python
    class D(B, C):
        def meth(self):                     # Redefine lower
            ...
            C.meth(self)                    # <== Pick C's method by calling
```

Such selections by assignment or call at mix-in points can effectively insulate your code
from this difference in class flavors. This applies only to the attributes you handle this
way, of course, but explicitly resolving the conflicts ensures that your code won’t vary
per Python version, at least in terms of attribute conflict selection. In other words, this
can serve as a *portability* technique for classes that may need to be run under both the
new-style and classic class models.

**Note**  
*Explicit is better than implicit—for method resolution too*: Even without
the classic/new-style class divergence, the explicit method resolution
technique shown here may come in handy in multiple inheritance scenarios 
in general. For instance, if you want part of a superclass on the
left and part of a superclass on the right, you might need to tell Python
which same-named attributes to choose by using explicit assignments
or calls in subclasses. We’ll revisit this notion in a “gotcha” at the end
of this section.

Also note that diamond inheritance patterns might be more problematic
in some cases than I’ve implied here (e.g., what if `B` and `C` both have
required constructors that call to the constructor in `A`?). Since such 
contexts are rare in real-world Python, we’ll defer this topic until we explore
the `super` built-in function near the end of this section; besides providing 
generic access to superclasses in single inheritance trees, `super` supports 
a cooperative mode for resolving conflicts in multiple inheritance
trees by ordering method calls per the MRO—assuming this order
makes sense in this context too!

#### Scope of search order change


In sum, by default, the diamond pattern is searched differently for classic and new-style
classes, and this is a non-backward-compatible change. Keep in mind, though, that this
change primarily affects diamond pattern cases of multiple inheritance; new-style class
inheritance works the same for most other inheritance tree structures. Further, it’s not
impossible that this entire issue may be of more theoretical than practical importance
—because the new-style search wasn’t significant enough to address until Python 2.2
and didn’t become standard until 3.0, it seems unlikely to impact most Python code.

Having said that, I should also note that even though you might not code diamond
patterns in classes you write yourself, because the implied `object` superclass is above
every root class in 3.X as we saw earlier, *every* case of multiple inheritance exhibits the
diamond pattern today. That is, in new-style classes, `object` automatically plays the
role that the class `A` does in the example we just considered. Hence the new-style MRO
search rule not only modifies logical semantics, but is also an important *performance optimization*—
it avoids visiting and searching the same class more than once, even the
automatic `object`.

Just as important, we’ve also seen that the implied `object` superclass in the new-style
model provides *default methods* for a variety of built-in operations, including the
`__str__` and `__repr__` display format methods. Run a `dir(object)` to see which methods
are provided. Without the new-style MRO search order, in multiple inheritance cases
the defaults in `object` would always override redefinitions in user-coded classes, unless
they were always made in the leftmost superclass. In other words, the new-style class
model itself makes using the new-style search order more critical!

For a more visual example of the implied `object` superclass in 3.X, and other examples
of diamond patterns created by it, see the `ListTree` class’s output in the `lister.py` example
in the preceding section, as well as the `classtree.py` tree walker example in [Chapter 29]()—
and the next section.

### More on the MRO: Method Resolution Order


To trace how new-style inheritance works by default, we can also use the new
`class.__mro__` attribute mentioned in the preceding section’s class lister examples—
technically a new-style extension, but useful here to explore a change. This attribute
returns a class’s *MRO*—the order in which inheritance searches classes in a new-style
class tree. This MRO is based on the C3 superclass linearization algorithm initially
developed in the Dylan programming language, but later adopted by other languages
including Python 2.3 and Perl 6.

#### The MRO algorithm


This book avoids a full description of the MRO algorithm deliberately, because many
Python programmers don’t need to care (this only impacts diamonds, which are relatively 
rare in real-world code); because it differs between 2.X and 3.X; and because the
details of the MRO are a bit too arcane and academic for this text. As a rule, this book
avoids formal algorithms and prefers to teach informally by example.

On the other hand, some readers may still have an interest in the formal theory behind
new-style MRO. If this set includes you, it’s described in full detail online; search
Python’s manuals and the Web for current MRO links. In short, though, the MRO
essentially works like this:

 1. List all the classes that an instance inherits from using the classic class’s *DFLR*
    lookup rule, and include a class multiple times if it’s visited more than once.

 2. Scan the resulting list for duplicate classes, removing all but the *last* occurrence of
    duplicates in the list.

The resulting MRO list for a given class includes the class, its superclasses, and all
higher superclasses up to the `object` root class at the top of the tree. It’s ordered such
that each class appears before its parents, and multiple parents retain the order in which
they appear in the `__bases__` superclass tuple.

Crucially, though, because common parents in *diamonds* appear only at the position
of their *last* visitation, lower classes are searched first when the MRO list is later used
by attribute inheritance. Moreover, each class is included and thus visited just once,
no matter how many classes lead to it.

We’ll see applications of this algorithm later in this section, including that in `super`—
a built-in that elevates the MRO to required reading if you wish to fully understand
how methods are dispatched by this call, should you choose to use it. As we’ll see,
despite its name, this call invokes the next class on the MRO, which might not be a
superclass at all.

#### Tracing the MRO


If you just want to see how Python’s new-style inheritance orders superclasses in general, 
though, new-style classes (and hence all classes in 3.X) have a *`class`*`.__mro__` attribute, 
which is a tuple giving the linear search order Python uses to look up attributes
in superclasses. Really, this attribute *is* the inheritance order in new-style classes, and
is often as much MRO detail as many Python users need.

Here are some illustrative examples, run in 3.X; for *diamond* inheritance patterns only,
the search is the new order we’ve been studying—*across* before up, per the MRO for
new-style classes always used in 3.X, and available as an option in 2.X:

In [ ]:
class A: pass
class B(A): pass # Diamonds: order differs for newstyle
class C(A): pass # Breadth-first across lower levels
class D(B, C): pass
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

For *nondiamonds*, though, the search is still as it has always been (albeit with an extra
`object` root)—to the top, and then to the right (a.k.a. *DFLR*, depth first and left to
right, the model used for all classic classes in 2.X):

```bash
    >>> class A: pass
    >>> class B(A): pass                # Nondiamonds: order same as classic
    >>> class C: pass                   # Depth first, then left to right
    >>> class D(B, C): pass
    >>> D.__mro__
    (<class '__main__.D'>, <class '__main__.B'>, <class '__main__.A'>,
    <class '__main__.C'>, <class 'object'>)
```

The MRO of the following tree, for example, is the same as the earlier diamond, per
DFLR:

In [ ]:
class A: pass
class B: pass                               # Another nondiamond: DFLR
class C(A): pass
class D(B, C): pass
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

Notice how the implied object superclass always shows up at the end of the MRO; as
we’ve seen, it’s added automatically above root (*topmost*) classes in new-style class trees
in 3.X (and optionally in 2.X):

In [ ]:
A.__bases__, B.__bases__, C.__bases__, D.__bases__

((object,), (object,), (__main__.A,), (__main__.B, __main__.C))

Technically, the implied `object` superclass always creates a diamond in multiple 
inheritance even if your classes do not—your classes are searched as before, but the 
new-style MRO ensures that `object` is visited last, so your classes can override its defaults:

In [ ]:
class X: pass
class Y: pass
class A(X): pass                            # Nondiamond: depth first then left to right
class B(Y): pass                            # Though implied "object" always forms a diamond
class D(A, B): pass
D.mro()

[__main__.D, __main__.A, __main__.X, __main__.B, __main__.Y, object]

In [ ]:
X.__bases__, Y.__bases__, A.__bases__, B.__bases__

((object,), (object,), (__main__.X,), (__main__.Y,))

The *`class`*`.__mro__` attribute is available only on new-style classes; it’s not present in
2.X unless classes derive from object. Strictly speaking, new-style classes also have a
*`class`*`.mro()` method used in the prior example for variety; it’s called at class instantiation 
time and its return value is a list used to initialize the `__mro__` attribute when the
class is created (the method is available for customization in metaclasses, described
later). You can also select MRO names if classes’ object displays are too detailed, though
this book usually shows the *objects* to remind you of their true form:

In [ ]:
D.mro() == list(D.__mro__)

True

In [ ]:
[cls.__name__ for cls in D.__mro__]

['D', 'A', 'X', 'B', 'Y', 'object']

However you access or display them, class MRO paths might be useful to resolve confusion, 
and in tools that must imitate Python’s inheritance search order. 
The next section shows the latter role in action.

### Example: Mapping Attributes to Inheritance Sources


As a prime MRO use case, we noted at the end of the prior section that class tree
climbers—such as the class tree lister mix-in we wrote there—might benefit from the
MRO. As coded, the tree lister gave the *physical* locations of attributes in a class tree.
However, by mapping the list of inherited attributes in a `dir` result to the linear MRO
sequence (or DFLR order for classic classes), such tools can more directly associate
attributes with the classes from which they are *inherited*—also a useful relationship for
programmers.

We won’t recode our tree lister here, but as a first major step, the following file,
`mapattrs.py`, implements tools that can be used to associate attributes with their 
inheritance source; as an added bonus, its `mapattrs` function demonstrates how inheritance 
actually searches for attributes in class tree objects, though the new-style MRO
is largely automated for us:

In [ ]:
"""
File mapattrs.py (3.X + 2.X)

Main tool: mapattrs() maps all attributes on or inherited by an
instance to the instance or class from which they are inherited.

Assumes dir() gives all attributes of an instance. To simulate
inheritance, uses either the class's MRO tuple, which gives the
search order for new-style classes (and all in 3.X), or a recursive
traversal to infer the DFLR order of classic classes in 2.X.

Also here: inheritance() gives version-neutral class ordering;
assorted dictionary tools using 3.X/2.7 comprehensions.
"""

import pprint

def trace(X, label='', end='\n'):
    print(label + pprint.pformat(X) + end)                          # Print nicely

def filterdictvals(D, V):
    """
    dict D with entries for value V removed.
    filterdictvals(dict(a=1, b=2, c=1), 1) => {'b': 2}
    """
    return {K: V2 for (K, V2) in D.items() if V2 != V}

def invertdict(D):
    """
    dict D with values changed to keys (grouped by values).
    Values must all be hashable to work as dict/set keys.
    invertdict(dict(a=1, b=2, c=1)) => {1: ['a', 'c'], 2: ['b']}
    """
    
    def keysof(V):
        return sorted(K for K in D.keys() if D[K] == V)
    return {V: keysof(V) for V in set(D.values())}

def dflr(cls):
    """
    Classic depth-first left-to-right order of class tree at cls.
    Cycles not possible: Python disallows on __bases__ changes.
    """
    here = [cls]
    for sup in cls.__bases__:
        here += dflr(sup)
    return here

def inheritance(instance):
    """
    Inheritance order sequence: new-style (MRO) or classic (DFLR)
    """
    if hasattr(instance.__class__, '__mro__'):
        return (instance,) + instance.__class__.__mro__
    else:
        return [instance] + dflr(instance.__class__)

def mapattrs(instance, withobject=False, bysource=False):
    """
    dict with keys giving all inherited attributes of instance,
    with values giving the object that each is inherited from.
    withobject: False=remove object built-in class attributes.
    bysource: True=group result by objects instead of attributes.
    Supports classes with slots that preclude __dict__ in instances.
    """
    attr2obj = {}
    inherits = inheritance(instance)
    for attr in dir(instance):
        for obj in inherits:
            if hasattr(obj, '__dict__') and attr in obj.__dict__:   # See slots
                attr2obj[attr] = obj
                break
    if not withobject:
        attr2obj = filterdictvals(attr2obj, object)
    return attr2obj if not bysource else invertdict(attr2obj)


if __name__ == '__main__':
    print('Classic classes in 2.X, new-style in 3.X')
    class A: attr1 = 1
    class B(A): attr2 = 2
    class C(A): attr1 = 3
    class D(B, C): pass
    I = D()
    print('Py=>%s' % I.attr1)                                       # Python's search == ours?
    trace(inheritance(I), 'INH\n')                                  # [Inheritance order]
    trace(mapattrs(I), 'ATTRS\n')                                   # Attrs => Source
    trace(mapattrs(I, bysource=True), 'OBJS\n')                     # Source => [Attrs]
    
    print('New-style classes in 2.X and 3.X')
    class A(object): attr1 = 1                                      # "(object)" optional in 3.X
    class B(A): attr2 = 2
    class C(A): attr1 = 3
    class D(B, C): pass
    I = D()
    print('Py=>%s' % I.attr1)
    trace(inheritance(I), 'INH\n')
    trace(mapattrs(I), 'ATTRS\n')
    trace(mapattrs(I, bysource=True), 'OBJS\n')

Classic classes in 2.X, new-style in 3.X
Py=>3
INH
(<__main__.D object at 0x000001998BB0C970>,
 <class '__main__.D'>,
 <class '__main__.B'>,
 <class '__main__.C'>,
 <class '__main__.A'>,
 <class 'object'>)

ATTRS
{'__dict__': <class '__main__.A'>,
 '__doc__': <class '__main__.D'>,
 '__module__': <class '__main__.D'>,
 '__weakref__': <class '__main__.A'>,
 'attr1': <class '__main__.C'>,
 'attr2': <class '__main__.B'>}

OBJS
{<class '__main__.A'>: ['__dict__', '__weakref__'],
 <class '__main__.B'>: ['attr2'],
 <class '__main__.C'>: ['attr1'],
 <class '__main__.D'>: ['__doc__', '__module__']}

New-style classes in 2.X and 3.X
Py=>3
INH
(<__main__.D object at 0x000001998BAE8310>,
 <class '__main__.D'>,
 <class '__main__.B'>,
 <class '__main__.C'>,
 <class '__main__.A'>,
 <class 'object'>)

ATTRS
{'__dict__': <class '__main__.A'>,
 '__doc__': <class '__main__.D'>,
 '__module__': <class '__main__.D'>,
 '__weakref__': <class '__main__.A'>,
 'attr1': <class '__main__.C'>,
 'attr2': <class '__m

This file assumes `dir` gives all an instance’s attributes. It maps each attribute in a `dir`
result to its source by scanning either the MRO order for new-style classes, or the DFLR
order for classic classes, searching each object’s namespace `__dict__` along the way.
For classic classes, the DFLR order is computed with a simple recursive scan. The net
effect is to simulate Python’s inheritance search in both class models.

This file’s self-test code applies its tools to the diamond multiple-inheritance trees we
saw earlier. It uses Python’s `pprint` library module to display lists and dictionaries nicely
—`pprint.pprint` is its basic call, and its `pformat` returns a print string. Run this on
Python 2.7 to see both classic DFLR and new-style MRO search orders; on Python 3.3,
the `object` derivation is unnecessary, and both tests give the same, new-style results.
Importantly, `attr1`, whose value is labeled with “`Py=>`” and whose name appears in
the results lists, is inherited from class `A` in classic search, but from class `C` in new-style
search.

As a larger application of these tools, the following is our inheritance simulator at work
in 3.3 on the preceding section’s `testmixin0.py` file’s test classes (I’ve deleted some 
built-in names here for space; as usual, run live for the whole list). Notice how `__X` pseudoprivate names are mapped to their defining classes, and how `ListInstance` appears in
the MRO *before* `object`, which has a `__str__` that would otherwise be chosen first—as
you’ll recall, mixing this method in was the whole point of the lister classes!

In [ ]:
# from mapattrs import trace, dflr, inheritance, mapattrs
from testmixin0 import Sub
I = Sub()                       # Sub inherits from Super and ListInstance roots
trace(dflr(I.__class__))        # 2.X search order: implied object before lister!

[<class 'testmixin0.Sub'>,
 <class 'testmixin0.Super'>,
 <class 'object'>,
 <class 'listinstance.ListInstance'>,
 <class 'object'>]



In [ ]:
trace(inheritance(I))           # 3.X (+ 2.X newstyle) search order: lister first

(<testmixin0.Sub object at 0x000001998BB1D4C0>,
 <class 'testmixin0.Sub'>,
 <class 'testmixin0.Super'>,
 <class 'listinstance.ListInstance'>,
 <class 'object'>)



In [ ]:
trace(mapattrs(I))

{'_ListInstance__attrnames': <class 'listinstance.ListInstance'>,
 '__dict__': <class 'testmixin0.Super'>,
 '__doc__': <class 'testmixin0.Sub'>,
 '__init__': <class 'testmixin0.Sub'>,
 '__module__': <class 'testmixin0.Sub'>,
 '__str__': <class 'listinstance.ListInstance'>,
 '__weakref__': <class 'testmixin0.Super'>,
 'data1': <testmixin0.Sub object at 0x000001998BB1D4C0>,
 'data2': <testmixin0.Sub object at 0x000001998BB1D4C0>,
 'data3': <testmixin0.Sub object at 0x000001998BB1D4C0>,
 'ham': <class 'testmixin0.Super'>,
 'spam': <class 'testmixin0.Sub'>}



In [ ]:
trace(mapattrs(I, bysource=True))

{<testmixin0.Sub object at 0x000001998BB1D4C0>: ['data1', 'data2', 'data3'],
 <class 'testmixin0.Sub'>: ['__doc__', '__init__', '__module__', 'spam'],
 <class 'testmixin0.Super'>: ['__dict__', '__weakref__', 'ham'],
 <class 'listinstance.ListInstance'>: ['_ListInstance__attrnames', '__str__']}



In [ ]:
trace(mapattrs(I, withobject=True))

{'_ListInstance__attrnames': <class 'listinstance.ListInstance'>,
 '__class__': <class 'object'>,
 '__delattr__': <class 'object'>,
 '__dict__': <class 'testmixin0.Super'>,
 '__dir__': <class 'object'>,
 '__doc__': <class 'testmixin0.Sub'>,
 '__eq__': <class 'object'>,
 '__format__': <class 'object'>,
 '__ge__': <class 'object'>,
 '__getattribute__': <class 'object'>,
 '__gt__': <class 'object'>,
 '__hash__': <class 'object'>,
 '__init__': <class 'testmixin0.Sub'>,
 '__init_subclass__': <class 'object'>,
 '__le__': <class 'object'>,
 '__lt__': <class 'object'>,
 '__module__': <class 'testmixin0.Sub'>,
 '__ne__': <class 'object'>,
 '__new__': <class 'object'>,
 '__reduce__': <class 'object'>,
 '__reduce_ex__': <class 'object'>,
 '__repr__': <class 'object'>,
 '__setattr__': <class 'object'>,
 '__sizeof__': <class 'object'>,
 '__str__': <class 'listinstance.ListInstance'>,
 '__subclasshook__': <class 'object'>,
 '__weakref__': <class 'testmixin0.Super'>,
 'data1': <testmixin0.Sub object 

Here’s the bit you might run if you want to label class objects with names inherited by
an instance, though you may want to filter out some built-in double-underscore names
for the sake of users’ eyesight!

In [ ]:
amap = mapattrs(I, withobject=True, bysource=True)
trace(amap)

{<testmixin0.Sub object at 0x000001998BB1D4C0>: ['data1', 'data2', 'data3'],
 <class 'testmixin0.Sub'>: ['__doc__', '__init__', '__module__', 'spam'],
 <class 'testmixin0.Super'>: ['__dict__', '__weakref__', 'ham'],
 <class 'listinstance.ListInstance'>: ['_ListInstance__attrnames', '__str__'],
 <class 'object'>: ['__class__',
                    '__delattr__',
                    '__dir__',
                    '__eq__',
                    '__format__',
                    '__ge__',
                    '__getattribute__',
                    '__gt__',
                    '__hash__',
                    '__init_subclass__',
                    '__le__',
                    '__lt__',
                    '__ne__',
                    '__new__',
                    '__reduce__',
                    '__reduce_ex__',
                    '__repr__',
                    '__setattr__',
                    '__sizeof__',
                    '__subclasshook__']}



Finally, and as both a follow-up to the prior section’s ruminations and segue to the
next section here, the following shows how this scheme works for class-based *slots*
attributes too. Because a class’s `__dict__` includes both normal class attributes and
individual entries for the instance attributes defined by its `__slots__` list, the slots 
attributes inherited by an instance will be correctly associated with the implementing
class from which they are acquired, even though they are not physically stored in the
instance’s `__dict__` itself:

In [ ]:
# mapattrs-slots.py: test __slots__ attribute inheritance

from mapattrs import mapattrs, trace

class A(object): __slots__ = ['a', 'b']; x = 1; y = 2
class B(A): __slots__ = ['b', 'c']
class C(A): x = 2
class D(B, C):
    z = 3
    def __init__(self): self.name = 'Bob';
    
I = D()
trace(mapattrs(I, bysource=True)) # Also: trace(mapattrs(I))

{<__main__.D object at 0x000001998BB73A40>: ['name'],
 <class '__main__.A'>: ['a', 'y'],
 <class '__main__.B'>: ['__slots__', 'b', 'c'],
 <class '__main__.D'>: ['__dict__',
                        '__doc__',
                        '__init__',
                        '__module__',
                        '__weakref__',
                        'z'],
 <class '__main__.C'>: ['x']}



For explicitly new-style classes like those in this file, the results are the same under both
2.7 and 3.3, though 3.3 adds an extra built-in name to the set. The attribute names here
reflect all those inherited by the instance from user-defined classes, even those implemented 
by slots defined at classes and stored in space allocated in the instance.

But we need to move ahead to understand the role of slots better—and understand why
`mapattrs` must be careful to check to see if a `__dict__` is present before fetching it!

Study this code for more insight. For the prior chapter’s tree lister, your next step might
be to index the `mapattrs` function’s `bysource=True` dictionary result to obtain an object’s
attributes during the tree sketch traversal, instead of (or perhaps in addition to?) its
current physical `__dict__` scan. You’ll probably need to use `getattr` on the instance to
fetch attribute values, because some may be implemented as slots or other “virtual”
attributes at their source classes, and fetching these at the class directly won’t return
the instance’s value. If I code anymore here, though, I’ll deprive readers of the remaining
fun, and the next section of its subject matter.

## New-Style Class Extensions


Beyond the changes described in the prior section (some of which, frankly, may seem
too academic and obscure to matter to many readers of this book), new-style classes
provide a handful of more advanced class tools that have more direct and practical
application—*slots*, *properties*, *descriptors*, and more. The following sections provide an
overview of each of these additional features, available for new-style class in Python
2.X and all classes in Python 3.X. Also in this extensions category are the `__mro__` attribute and the `super` call, both covered elsewhere—the former in the previous section
to explore a change, and the latter postponed until chapter end to serve as a larger case
study.

### Slots: Attribute Declarations


By assigning a sequence of string attribute names to a special `__slots__` class attribute,
we can enable a new-style class to both limit the set of legal attributes that instances of
the class will have, and optimize memory usage and possibly program speed. As we’ll
find, though, slots should be used only in applications that clearly warrant the added
complexity. They will complicate your code, may complicate or break code you may
use, and require universal deployment to be effective.



#### Slot basics


To use slots, assign a sequence of string names to the special `__slots__` variable and
attribute at the top level of a `class` statement: only those names in the `__slots__` list
can be assigned as instance attributes. However, like all names in Python, instance
attribute names must still be assigned before they can be referenced, even if they’re
listed in `__slots__`:

In [ ]:
class limiter(object):
    __slots__ = ['age', 'name', 'job']

x = limiter()
x.age                                           # Must assign before use

AttributeError: age

In [ ]:
x.age = 40                                      # Looks like instance data
x.age

40

In [ ]:
x.ape = 1000                                    # Illegal: not in __slots__

AttributeError: 'limiter' object has no attribute 'ape'

This feature is envisioned as both a way to catch typo errors like this (assignments to
illegal attribute names not in `__slots__` are detected) as well as an optimization mechanism.

Allocating a namespace dictionary for every instance object can be expensive in terms
of memory if many instances are created and only a few attributes are required. To save
space, instead of allocating a dictionary for each instance, Python reserves just enough
space in each *instance* to hold a value for each slot attribute, along with inherited 
attributes in the common class to manage slot access. This might additionally speed
execution, though this benefit is less clear and might vary per program, platform, and
Python.

Slots are also something of a major break with Python’s core dynamic nature, which
dictates that any name may be created by assignment. In fact, they imitate C++ for
efficiency at the expense of flexibility, and even have the potential to *break* some programs. 
As we’ll see, slots also come with a plethora of special-case usage rules. Per
Python’s own manual, they should *not* be used except in clearly warranted cases—they
are difficult to use correctly, and are, to quote the manual:

```
    best reserved for rare cases where there are large numbers of instances 
    in a memory-critical application.
```

In other words, this is yet another feature that should be used only if clearly warranted.
Unfortunately, slots seem to be showing up in Python code much more often than they
should; their obscurity seems to be a draw in itself. As usual, knowledge is your best
ally in such things, so let’s take a quick look here.

#### Slots and namespace dictionaries


Potential benefits aside, slots can complicate the class model—and code that relies on
it—substantially. In fact, some instances with slots may not have a `__dict__` attribute
namespace dictionary at all, and others will have data attributes that this dictionary
does not include. To be clear: this is a *major incompatibility* with the traditional class
model—one that can complicate any code that accesses attributes generically, and may
even cause some programs to fail altogether.

For instance, programs that list or access instance attributes by name string may need
to use more storage-neutral interfaces than `__dict__` if slots may be used. Because an
instance’s data may include class-level names such as slots—either in addition to or
instead of namespace dictionary storage—both attribute sources may need to be queried 
for completeness.

Let’s see what this means in terms of code, and explore more about slots along the way.
First off, when slots are used, instances do not normally have an attribute dictionary
—instead, Python uses the class *descriptors* feature introduced ahead to allocate and
manage space reserved for slot attributes in the instance. In Python 3.X, and in 2.X for
new-style classes derived from object:

In [ ]:
class C:                                    # Requires "(object)" in 2.X only
    __slots__ = ['a', 'b']                  # __slots__ means no __dict__ by default

X = C()
X.a = 1
X.a

1

In [ ]:
X.__dict__

AttributeError: 'C' object has no attribute '__dict__'

However, we can still fetch and set slot-based attributes by name string using 
storage-neutral tools such as `getattr` and `setattr` (which look beyond the instance `__dict__`
and thus include class-level names like slots) and `dir` (which collects all inherited names
throughout a class tree):

In [ ]:
getattr(X, 'a')

1

In [ ]:
setattr(X, 'b',  2)

In [ ]:
'a' in dir(X), 'b' in dir(X)                    # And dir() finds slot attributes too

(True, True)

Also keep in mind that without an attribute namespace dictionary, it’s not possible to
assign new names to instances that are not names in the slots list:

In [ ]:
class D:  # Use D(object) for same result in 2.X
    __slots__ = ['a', 'b']
    def __init__(self):
        self.d = 4 # Cannot add new names if no __dict__

X = D()

AttributeError: 'D' object has no attribute 'd'

We can still accommodate extra attributes, though, by including `__dict__` explicitly in
`__slots__`, in order to create an attribute namespace dictionary too:

In [ ]:
class D:
    __slots__ = ['a', 'b', '__dict__']          # Name __dict__ to include one too
    c = 3                                       # Class attrs work normally
    def __init__(self):
        self.d = 4                              # d stored in __dict__, a is a slot

X = D()
X.d, X.c

(4, 3)

In [ ]:
X.a                                     # All instance attrs undefined until assigned

AttributeError: a

In [ ]:
X.a = 1
X.b = 2

In this case, *both* storage mechanisms are used. This renders `__dict__` too limited for
code that wishes to treat slots as instance data, but generic tools such as `getattr` still
allow us to process both storage forms as a single set of attributes:

In [ ]:
X.__dict__                  # Some objects have both __dict__ and slot names 
                            # getattr() can fetch either type of attr

{'d': 4}

In [ ]:
X.__slots__

['a', 'b', '__dict__']

In [ ]:
getattr(X, 'a'), getattr(X, 'c'), getattr(X, 'd')       # Fetches all 3 forms

(1, 3, 4)

Because `dir` also returns all *inherited* attributes, though, it might be too broad in some
contexts; it also includes class-level methods, and even all `object` defaults. Code that
wishes to list *just* instance attributes may in principle still need to allow for both storage
forms explicitly. We might at first naively code this as follows:

In [ ]:
for attr in list(X.__dict__) + X.__slots__:             # Wrong...
    print(attr, '=>', getattr(X, attr))

d => 4
a => 1
b => 2
__dict__ => {'d': 4}


Since either can be omitted, we may more correctly code this as follows, using 
`getattr` to allow for defaults—a noble but nonetheless inaccurate approach, as the next
section will explain:

In [ ]:
for attr in list(getattr(X, '__dict__', [])) + getattr(X, '__slots__', []):
    print(attr, '=>', getattr(X, attr))

d => 4
a => 1
b => 2
__dict__ => {'d': 4}


#### Multiple `__slot__` lists in superclasses


The preceding code works in this specific case, but in general it’s *not entirely accurate*. 
Specifically, this code addresses only slot names in the *lowest* `__slots__` attribute
inherited by an instance, but slot lists may appear more than once in a class tree. That
is, a name’s absence in the lowest `__slots__` list does not preclude its existence in a
higher `__slots__`. Because slot names become class-level attributes, instances acquire
the union of all slot names anywhere in the tree, by the normal inheritance rule:

In [ ]:
class E:
    __slots__ = ['c', 'd']              # Superclass has slots

class D(E):
    __slots__ = ['a', '__dict__']       # But so does its subclass

X = D()
X.a = 1; X.b = 2; X.c = 3               # The instance is the union (slots: a, c)
X.a, X.c

(1, 3)

Inspecting just the inherited slots list won’t pick up slots defined higher in a class tree:

In [ ]:
E.__slots__, D.__slots__                    # But slots are not concatenated

(['c', 'd'], ['a', '__dict__'])

In [ ]:
X.__slots__                         # Instance inherits *lowest* __slots__

['a', '__dict__']

In [ ]:
X.__dict__                          # And has its own an attr dict

{'b': 2}

In [ ]:
for attr in list(getattr(X, '__dict__', [])) + getattr(X, '__slots__', []):
    print(attr, '=>', getattr(X, attr))             # Other superclass slots missed!

b => 2
a => 1
__dict__ => {'b': 2}


In [ ]:
print(dir(X))                          # But dir() includes all slot names

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', 'a', 'b', 'c', 'd']


In other words, in terms of listing instance attributes generically, one `__slots__` isn’t
always enough—they are potentially subject to the full inheritance search procedure.
See the earlier `mapattrs-slots.py` for another example of slots appearing in multiple
superclasses. If multiple classes in a class tree have their own `__slots__` attributes,
generic programs must develop other policies for listing attributes—as the next section
explains.

#### Handling slots and other “virtual” attributes generically


At this point, you may wish to review the discussion of slots policy options at the
coverage of the `lister.py` display mix-in classes near the end of the preceding section—
a prime example of why generic programs may need to care about slots. Such tools that
attempt to list instance data attributes generically must account for slots, and perhaps
other such “virtual” instance attributes like `properties` and `descriptors` discussed ahead
—names that similarly reside in classes but may provide attribute values for instances
on request. Slots are the most data-centric of these, but are representative of a larger
category.

Such attributes require inclusive approaches, special handling, or general avoidance—
the latter of which becomes unsatisfactory as soon as any programmer uses slots in
subject code. Really, class-level instance attributes like slots probably necessitate a 
redefinition of the term *instance data*—as locally stored attributes, the union of all 
inherited attributes, or some subset thereof.

For example, some programs might classify slot names as attributes of *classes* instead
of instances; these attributes do not exist in instance namespace dictionaries, after all.
Alternatively, as shown earlier, programs can be more inclusive by relying on `dir` to
fetch all inherited attribute names and `getattr` to fetch their corresponding values for
the instance—without regard to their physical location or implementation. If you must
support slots as instance data, this is likely the most robust way to proceed:

In [ ]:
class Slotful:
    __slots__ = ['a', 'b', '__dict__']
    def __init__(self, data):
        self.c = data
    
I = Slotful(3)
I.a, I.b = 1, 2
I.a, I.b, I.c                           # Normal attribute fetch

(1, 2, 3)

In [ ]:
I.__dict__                              # Both __dict__ and slots storage

{'c': 3}

In [ ]:
[x for x in dir(I) if not x.startswith('__')]

['a', 'b', 'c']

In [ ]:
I.__dict__['c']                         # __dict__ is only one attr source

3

In [ ]:
getattr(I, 'c'), getattr(I, 'a')        # dir+getattr is broader than __dict__
                                        # applies to slots, properties, descrip

(3, 1)

In [ ]:
for a in (x for x in dir(I) if not x.startswith('__')):
    print(a, getattr(I, a))

a 1
b 2
c 3


Under this `dir`/`getattr` model, you can still map attributes to their inheritance sources,
and filter them more selectively by source or type if needed, by scanning the *MRO*—
as we did earlier in both `mapattrs.py` and its application to slots in `mapattrs-slots.py`.
As an added bonus, such tools and policies for handling slots will potentially apply
automatically to *properties* and *descriptors* too, though these attributes are more 
explicitly computed values, and less obviously instance-related data than slots.

Also keep in mind that this is not just a tools issue. Class-based instance attributes like
slots also impact the traditional coding of the `__setattr__` operator overloading method
we met in [Chapter 30](). Because slots and some other attributes are not stored in the
instance `__dict__`, and may even imply its *absence*, new-style classes must instead 
generally run attribute assignments by routing them to the `object` superclass. In practice,
this may make this method fundamentally different in some classic and new-style
classes.

#### Slot usage rules


Slot declarations can appear in multiple classes in a class tree, but when they do they
are subject to a number of constraints that are somewhat difficult to rationalize unless
you understand the implementation of slots as class-level *descriptors* for each slot name
that are inherited by the instances where the managed space is reserved (descriptors
are an advanced tool we’ll study in detail in the last part of this book):

  + *Slots in subs are pointless when absent in supers*: If a subclass inherits from a 
    superclass without a `__slots__`, the instance `__dict__` attribute created for the 
    superclass will always be accessible, making a `__slots__` in the subclass largely 
    pointless. The subclass still manages its slots, but doesn’t compute their values in any
    way, and doesn’t avoid a dictionary—the main reason to use slots.

  + *Slots in supers are pointless when absent in subs*: Similarly, because the meaning of
    a `__slots__` declaration is limited to the class in which it appears, subclasses will
    produce an instance `__dict__` if they do not define a `__slots__`, rendering a
    `__slots__` in a superclass largely pointless.

  + *Redefinition renders super slots pointless*: If a class defines the same slot name as a
    superclass, its redefinition hides the slot in the superclass per normal inheritance.
    You can access the version of the name defined by the superclass slot only by
    fetching its descriptor directly from the superclass.

  + *Slots prevent class-level defaults*: Because slots are implemented as class-level 
    descriptors (along with per-instance space), you cannot use class attributes of the
    same name to provide defaults as you can for normal instance attributes: assigning
    the same name in the class overwrites the slot descriptor.

  + *Slots and `__dict__`*: As shown earlier, `__slots__` preclude both an instance
    `__dict__` and assigning names not listed, unless `__dict__` is listed explicitly too.

We’ve already seen the last of these in action, and the earlier `mapattrs-slots.py` illustrates
the third. It’s easy to demonstrate how the new rules here translate to actual code—
most crucially, a namespace dictionary is created when any class in a tree omits slots,
thereby negating the memory optimization benefit:

In [ ]:
class C: pass                           # Bullet 1: slots in sub but not super
class D(C): __slots__ = ['a']           # Makes instance dict for nonslots
X = D()                                 # But slot name still managed in class
X.a = 1; X.b = 2
X.__dict__

{'b': 2}

In [ ]:
D.__dict__.keys()

dict_keys(['__module__', '__slots__', 'a', '__doc__'])

In [ ]:
class C: __slots__ = ['a']              # Bullet 2: slots in super but not sub
class D(C): pass                        # Makes instance dict for nonslots
X = D()                                 # But slot name still managed in class
X.a = 1; X.b = 2
X.__dict__

{'b': 2}

In [ ]:
C.__dict__.keys()

dict_keys(['__module__', '__slots__', 'a', '__doc__'])

In [ ]:
class C: __slots__ = ['a']              # Bullet 3: only lowest slot accessible
class D(C): __slots__ = ['a']
class C: __slots__ = ['a']; a = 99      # Bullet 4: no class-level defaults

ValueError: 'a' in __slots__ conflicts with class variable

In other words, besides their program-breaking potential, slots essentially require 
*both universal and careful deployment* to be effective—because slots do not compute values
dynamically like properties (coming up in the next section), they are largely pointless
unless each class in a tree uses them and is cautious to define only new slot names not
defined by other classes. It’s an *all-or-nothing* feature—an unfortunate property shared
by the `super` call discussed ahead:

In [ ]:
class C: __slots__ = ['a']              # Assumes universal use, differing names
class D(C): __slots__ = ['b']
X = D()
X.a = 1; X.b = 2
X.__dict__

AttributeError: 'D' object has no attribute '__dict__'

In [ ]:
C.__dict__.keys(), D.__dict__.keys()

(dict_keys(['__module__', '__slots__', 'a', '__doc__']),
 dict_keys(['__module__', '__slots__', 'b', '__doc__']))

Such rules—among others regarding *weak references* omitted here for space—are part
of the reason slots are not generally recommended, except in pathological cases where
their space reduction is significant. Even then, their potential to complicate or break
code should be ample cause to carefully consider the tradeoffs. Not only must they be
spread almost *neurotically* throughout a framework, they may also break tools you rely
on.

##### Example impacts of slots: `ListTree` and mapattrs


As a more realistic example of slots’ effects, due to the first bullet in the prior section,
[Chapter 31]()’s `ListTree` class does *not fail* when mixed in to a class that defines
`__slots__`, even though it scans instance namespace dictionaries. The lister class’s own
lack of slots is enough to ensure that the instance will still have a `__dict__`, and hence
not trigger an exception when fetched or indexed. For example, both of the following
display without error—the second also allows names not in the slots list to be assigned
as instances attributes, including any required by the superclass:

```python
    class C(ListTree): pass
    X = C() # OK: no __slots__ used
    print(X)

    class C(ListTree): __slots__ = ['a', 'b']   # OK: superclass produces __dict__
    X = C()
    X.c = 3
    print(X)                                    # Displays c at X, a and b at C
```

The following classes display correctly as well—*any* nonslot class like `ListTree` 
generates an instance `__dict__`, and can thus safely assume its presence:

```python
    class A: __slots__ = ['a']                  # Both OK by bullet 1 above
    class B(A, ListTree): pass

    class A: __slots__ = ['a']
    class B(A, ListTree): __slots__ = ['b']     # Displays b at B, a at A
```

Although it renders subclass slots pointless, this is a positive side effect for tools classes
like `ListTree` (and its [Chapter 28]() predecessor). In general, though, some tools might
need to catch exceptions when `__dict__` is absent or use a `hasattr` or `getattr` to test or
provide defaults if slot usage may preclude a namespace dictionary in instance objects
inspected.

For example, you should now be able to understand why the `mapattrs.py` program
earlier in this section must check for the presence of a `__dict__` before fetching it—
instance objects created from classes with `__slots__` won’t have one. In fact, if we use
the highlighted alternative line in the following, the `mapattrs` function fails with an
exception when attempting to look for an attribute name in the instance at the front of
the inheritance path sequence:

```python
    def mapattrs(instance, withobject=False, bysource=False):
        for attr in dir(instance):
            for obj in inherits:
                if attr in obj.__dict__: # May fail if __slots__ used

    >>> class C: __slots__ = ['a']
    >>> X = C()
    >>> mapattrs(X)
    AttributeError: 'C' object has no attribute '__dict__'
```

Either of the following works around the issue, and allows the tool to support slots—
the first provides a default, and the second is more verbose but seems marginally more
explicit in its intent:

```python
    if attr in getattr(obj, '__dict__', {}):
    if hasattr(obj, '__dict__') and attr in obj.__dict__:
```

As mentioned earlier, some tools may benefit from mapping `dir` results to objects in
the MRO this way, instead of scanning an instance `__dict__` in general—without this
more inclusive approach, attributes implemented by class-level tools like slots won’t
be reported as instance data. Even so, this doesn’t necessarily excuse such tools from
allowing for a missing `__dict__` in the instance too!

#### What about slots speed?


Finally, while slots primarily optimize memory use, their speed impact is less clear-cut.
Here’s a simple test script using the `timeit` techniques we studied in [Chapter 21](). For
both the slots and nonslots (instance dictionary) storage models, it makes 1,000 instances, assigns and fetches 4 attributes on each, and repeats 1,000 times—for both
models taking the best of 3 runs that each exercise a total of 8M attribute operations:

In [ ]:
# File slots-test.py
from __future__ import print_function
import timeit
base = """
Is = []
for i in range(1000):
    X = C()
    X.a = 1; X.b = 2; X.c = 3; X.d = 4
    t = X.a + X.b + X.c + X.d
    Is.append(X)
"""

stmt = """
class C:
    __slots__ = ['a', 'b', 'c', 'd']
""" + base
print('Slots =>', end=' ')
print(min(timeit.repeat(stmt, number=1000, repeat=3)))

stmt = """
class C:
    pass
""" + base
print('Nonslots=>', end=' ')
print(min(timeit.repeat(stmt, number=1000, repeat=3)))

Slots => 1.0686648000000787
Nonslots=> 1.2949225000002116


At least on this code, on my laptop, and in my installed versions (Python 3.3 and 2.7),
the best times imply that slots are slightly quicker in 3.X and a wash in 2.X, though this
says little about memory space, and is prone to change arbitrarily in the future:

In [ ]:
!python slots-test.py

Slots => 1.0692428999999999
Nonslots=> 1.2588049999999997


For more on slots in general, see the Python standard manual set. Also watch for the
`Private` decorator case study of [Chapter 39]()—an example that naturally allows for 
attributes based on both `__slots__` and `__dict__` storage, by using delegation and 
storage-neutral accessor tools like `getattr`.

### Properties: Attribute Accessors


Our next new-style extension is *properties*—a mechanism that provides another way
for new-style classes to define methods called automatically for access or assignment
to instance attributes. This feature is similar to properties (a.k.a. “getters” and “setters”)
in languages like Java and C#, but in Python is generally best used sparingly, as a way
to add accessors to attributes *after the fact* as needs evolve and warrant. Where needed,
though, properties allow attribute values to be computed dynamically without requiring 
method calls at the point of access.

Though properties cannot support generic attribute routing goals, at least for specific
attributes they are an alternative to some traditional uses of the `__getattr__` and
`__setattr__` overloading methods we first studied in [Chapter 30](). Properties have a
similar effect to these two methods, but by contrast incur an extra method call only for
accesses to names that require dynamic computation—other nonproperty names are
accessed normally with no extra calls. Although `__getattr__` is invoked only for undefined 
names, the `__setattr__` method is instead called for assignment to *every* attribute.

Properties and slots are related too, but serve different goals. Both implement instance
attributes that are not physically stored in instance namespace dictionaries—a sort of
“virtual” attribute—and both are based on the notion of class-level attribute *descriptors*. 
In contrast, slots manage instance storage, while properties intercept access and
compute values arbitrarily. Because their underlying descriptor implementation tool is
too advanced for us to cover here, properties and descriptors both get full treatment in
[Chapter 38]().

#### Property basics


As a brief introduction, though, a property is a type of object assigned to a class attribute
name. You generate a property by calling the `property` built-in function, passing in up
to three accessor methods—handlers for get, set, and delete operations—as well as an
optional docstring for the property. If any argument is passed as `None` or omitted, that
operation is not supported.

The resulting property object is typically assigned to a name at the top level of a
`class` statement (e.g., `name=property()`), and a special `@` syntax we’ll meet later is 
available to automate this step. When thus assigned, later accesses to the class property
name itself as an object attribute (e.g., `obj.name`) are automatically routed to one of the
accessor methods passed into the `property` call.

For example, we’ve seen how the `__getattr__` operator overloading method allows
classes to intercept undefined attribute references in both classic and new-style classes:

In [ ]:
class operators:
    def __getattr__(self, name):
        if name == 'age':
            return 40
        else:
            raise AttributeError(name)

x = operators()
x.age                                       # Runs __getattr__

40

In [ ]:
x.name                                      # Runs __getattr__

AttributeError: name

Here is the same example, coded with properties instead; note that properties are
available for all classes but require the new-style `object` derivation in 2.X to work 
properly for intercepting attribute *assignments* (and won’t complain if you forget this—but
will silently overwrite your property with the new data!):

In [ ]:
class properties(object):                       # Need object in 2.X for setters
    def getage(self):
        return 40
    age = property(getage, None, None, None)    # (get, set, del, docs), or use @

x = properties()
x.age                                           # Runs getage

40

In [ ]:
x.name                                          # Normal fetch

AttributeError: 'properties' object has no attribute 'name'

For some coding tasks, properties can be less complex and quicker to run than the
traditional techniques. For example, when we add attribute *assignment* support, 
properties become more attractive—there’s less code to type, and no extra method calls are
incurred for assignments to attributes we don’t wish to compute dynamically:

In [ ]:
class properties(object):                   # Need object in 2.X for setters
    def getage(self):
        return 40
    def setage(self, value):
        print('set age: %s' % value)
        self._age = value
    age = property(getage, setage, None, None)

x = properties()
x.age                                       # Runs getage

40

In [ ]:
x.age = 42                                  # Runs setage

set age: 42


In [ ]:
x._age                                      # Normal fetch: no getage call

42

In [ ]:
x.age                                       # Runs getage

40

In [ ]:
x.job = 'trainer'                           # Normal assign: no setage call
x.job                                       # Normal fetch: no getage call

'trainer'

The equivalent class based on operator overloading incurs extra method calls for 
assignments to attributes not being managed and needs to route attribute assignments
through the attribute dictionary to avoid loops (or, for new-style classes, to the
`object` superclass’s `__setattr__` to better support “virtual” attributes such as slots and
properties coded in other classes):

In [ ]:
class operators:
    def __getattr__(self, name):                # On undefined reference
        if name == 'age':
            return 40
        else:
            raise AttributeError(name)
    def __setattr__(self, name, value):         # On all assignments
        print('set: %s %s' % (name, value))
        if name == 'age':
            self.__dict__['_age'] = value       # Or object.__setattr__()
        else:
            self.__dict__[name] = value

x = operators()
x.age                                           # Runs __getattr__

40

In [ ]:
x.age = 41                                      # Runs __setattr__

set: age 41


In [ ]:
x._age                                          # Defined: no __getattr__ call

41

In [ ]:
x.age                                           # Runs __getattr__

40

In [ ]:
x.job = 'trainer'                               # Runs __setattr__ again

set: job trainer


In [ ]:
x.job                                           # Defined: no __getattr__ call

'trainer'

Properties seem like a win for this simple example. However, some applications of
`__getattr__` and `__setattr__` still require more dynamic or generic interfaces than
properties directly provide.

For example, in many cases the set of attributes to be supported cannot be determined
when the class is coded, and may not even exist in any tangible form (e.g., when
*delegating* arbitrary attribute references to a wrapped/embedded object generically). In
such contexts, a generic `__getattr__` or a `__setattr__` attribute handler with a passed-in 
attribute name is usually preferable. Because such generic handlers can also support
simpler cases, properties are often an optional and redundant extension—albeit one
that may avoid extra calls on assignments, and one that some programmers may prefer
when applicable.

For more details on both options, stay tuned for [Chapter 38]() in the final part of this
book. As we’ll see there, it’s also possible to code properties using the `@` symbol 
*function decorator syntax*—a topic introduced later in this section, and an equivalent and automatic alternative to manual assignment in the class scope:

```python
    class properties(object):
        @property                       # Coding properties with decorators: ahead
        def age(self):
            ...
        @age.setter
        def age(self, value):
            ...
```

To make sense of this decorator syntax, though, we must move ahead.

### `__getattribute__` and Descriptors: Attribute Tools


Also in the class extensions department, the `__getattribute__` operator overloading
method, available for new-style classes only, allows a class to intercept *all* attribute
references, not just undefined references. This makes it more potent than its `__getattr__` cousin we used in the prior section, but also trickier to use—it’s prone to loops
much like `__setattr__`, but in different ways.

For more specialized attribute interception goals, in addition to properties and operator
overloading methods, Python supports the notion of attribute *descriptors*—classes with
`__get__` and `__set__` methods, assigned to class attributes and inherited by instances,
that intercept read and write accesses to specific attributes. As a preview, here’s one of
the simplest descriptors you’re likely to encounter:

In [ ]:
class AgeDesc(object):
    def __get__(self, instance, owner): return 40
    def __set__(self, instance, value): instance._age = value

class descriptors(object):
    age = AgeDesc()

x = descriptors()
x.age                               # Runs AgeDesc.__get__

40

In [ ]:
x.age = 42                          # Runs AgeDesc.__set__
x._age                              # Normal fetch: no AgeDesc call

42

Descriptors have access to state in instances of themselves as well as their client class,
and are in a sense a more general form of properties; in fact, properties are a simplified
way to define a specific type of descriptor—one that runs functions on access. 
Descriptors are also used to implement the slots feature we met earlier, and other Python
tools.

Because `__getattribute__` and descriptors are too substantial to cover well here, we’ll
defer the rest of their coverage, as well as much more on properties, to [Chapter 38]() in
the final part of this book. We’ll also employ them in examples in [Chapter 39]() and study
how they factor into inheritance in [Chapter 40]().

## Other Class Changes and Extensions


As mentioned, we’re also postponing coverage of the super built-in—an additional
major new-style class extension that relies on its MRO—until the end of this ssection.
Before we get there, though, we’re going to explore additional class-related changes
and extensions that are not necessarily bound to new-style classes, but were introduced
at roughly the same time: static and class methods, decorators, and more.

Many of the changes and feature additions of new-style classes integrate with the notion
of subclassable types mentioned earlier in this ssection, because subclassable types and
new-style classes were introduced in conjunction with a merging of the type/class 
dichotomy in Python 2.2 and beyond. As we’ve seen, in 3.X, this merging is complete:
classes are now types, and types are classes, and Python classes today still reflect both
that conceptual merging and its implementation.

Along with these changes, Python also grew a more coherent and generalized protocol
for coding *metaclasses*—classes that subclass the `type` object, intercept class creation
calls, and may provide behavior acquired by classes. Accordingly, they provide a 
well-defined hook for management and augmentation of class objects. They are also an
advanced topic that is optional for most Python programmers, so we’ll postpone further
details here. We’ll glimpse metaclasses again later in this ssection in conjunction with
class decorators—a feature whose roles often overlap—but we’ll postpone their full
coverage until [Chapter 40](), in the final part of this book. For our purpose here, let’s
move on to a handful of additional class-related extensions.